In [43]:
import numpy as np
import heapq
import pandas as pd

In [44]:
class fileClass:
    f = None
    array = []
    data_dict={}
    sorted_dict={}
    tree=None
    def __init__(self,path):
        self.f = open(path, "r")
        
        self.array = []
        self.data_dict={}
        self.sorted_dict={}
        self.tree=None
    def readFile(self):
        line=" "
        while line!='':
            line = self.f.read(1)
            self.array.append(line)
            if line in self.data_dict.keys():
                self.data_dict[line]+=1
                continue
            self.data_dict[line]=1
        self.sort_dict()
    def sort_dict(self):
        self.sorted_dict = dict(sorted(self.data_dict.items(),key=lambda x:x[1]))

 

In [45]:
class Node:
    left=None
    right=None
    parent=None
    val=0
    let=None
    isRight=''
    def __init__(self,val,parent=None,isRight=0,let=None):
        self.parent=parent
        self.val=val
        self.let=let
        self.isRight=isRight
    def __lt__(self,other):
        return self.val<=other.val
    def __str__(self):
        if self.let!=None:
            return "Node with value="+str(self.val)+" and letter="+self.let
        else:
            return "Node with value="+str(self.val)

In [46]:
class Tree:
    root=None
    new_codes={}
    def __init__(self,root):
        self.root=root
        
    def dfs(self,goal):
        frontier = [] #using a stack data structure, to perform a last 
                        #in first out protocol to go deep inside the tree before going the sibling node
        frontier.append(self.root)
        explored = []
        while frontier:
            node = frontier.pop()
            explored.append(node)
            if node.let==goal:
                return node
            if node.left != None and node.left not in explored:
                frontier.append(node.left)
            if node.right != None and node.right not in explored:
                frontier.append(node.right)
                
        return None
        
            
        #return None
    def get_ancestral_chain(self,node):
        current=node
        chain=[current]
        while current.parent!=None:
            current=current.parent
            chain.append(current)
        return chain
    def construct_new_codes(self,old_codes):
        for i in list(old_codes.keys()):
            node=self.dfs(goal=i)
            chain=self.get_ancestral_chain(node)
            code=""
            while(len(chain)>0):
                next_el=chain.pop()
                if(next_el.isRight!=None):
                    code+=str(next_el.isRight)        
            self.new_codes[i]=code
            #print(code)
            
        

In [47]:
class Huffman:
    file=None
    def __init__(self,file):
        self.file=file
    def construct_file_tree(self):
        min_heap=[]
        parent=None
        for i in file.sorted_dict.keys():
            node=Node(val=self.file.sorted_dict[i],let=i)
            heapq.heappush(min_heap,node)
        while(len(min_heap)>1):
            node1=heapq.heappop(min_heap)
            node2=heapq.heappop(min_heap)
            parent=Node(val=node1.val+node2.val)
            node1.parent=node2.parent=parent
            if node1<node2:
                node1.isRight='0'
                node2.isRight='1'
                parent.left=node1
                parent.right=node2
            else:
                node1.isRight='1'
                node2.isRight='0'
                parent.left=node2
                parent.right=node1
            heapq.heappush(min_heap,parent)
        parent.isRight=None
        self.file.tree=Tree(parent)            

In [48]:
#cd Huffman_project

In [49]:
file=fileClass("file.txt")
file.readFile()
file.data_dict.pop('')
file.sorted_dict.pop('')

1

In [50]:
huf=Huffman(file)

In [51]:
huf.construct_file_tree()

In [52]:
n=file.tree.dfs(goal='a')


In [53]:
chain=file.tree.get_ancestral_chain(n)
print(chain[1])

Node with value=100


In [54]:
#old_codes={'a':"000",'b':"001",'c':"010",'d':"011",'e':"100",'f':"101"}

In [55]:
#old_codes

In [56]:
#file.tree.construct_new_codes(old_codes)

In [57]:
#file.tree.new_codes

In [58]:
huf.file.sorted_dict

{'f': 5, 'e': 9, 'c': 12, 'b': 13, 'd': 16, 'a': 45}

In [59]:
#huf.file.tree.construct_new_codes(huf.file.data_dict)

In [60]:
huf.file.tree.new_codes

{}

In [61]:
huf.file.sorted_dict

{'f': 5, 'e': 9, 'c': 12, 'b': 13, 'd': 16, 'a': 45}

In [62]:
def compress(file):
    file.tree.construct_new_codes(file.data_dict)
    compressed_file = open("compressed_file.bin",'w')
    
    flag_array = [] #array to know which characters are already printed
    
    encoded_bits = ''
    
    character_frame = pd.DataFrame(columns = ['Byte','Code','New Code'])
    
    for character in file.array:
        if(character==''):
            break
        encoded_bits = encoded_bits + str(file.tree.new_codes[character])
        
        if(character not in flag_array):
            character_frame=character_frame.append({"Byte":character,"Code":str(bin(ord(character)))[2:],
                                    "New Code":file.tree.new_codes[character]},ignore_index=True)
                        
            flag_array.append(character)
    
    print(character_frame)
    
    if len(encoded_bits)%8!=0:
        for i in range(8-len(encoded_bits)%8):
            encoded_bits + '0'                   #padding
    
    encoded_bits = encoded_bits + "{0:08b}".format( (8-len(encoded_bits)%8) )
    
    byte = ''
    for bit in encoded_bits:
        byte = byte + bit
        if(len(byte)>8):
            compressed_file.write(str(int(byte[:8],2)))
            byte = bit
            
    compressed_file.close()

In [63]:
compress(huf.file)

  Byte     Code New Code
0    a  1100001        0
1    b  1100010      101
2    c  1100011      100
3    d  1100100      111
4    e  1100101     1101
5    f  1100110     1100


In [32]:
def decompress(compressed_file,bit_codes_dict):
    decompressed_file = open("decompressed_file.txt",'w')
    encoded_bits = compressed_file.readline()
    decoded_character_code = ''
    print(encoded_bits)
    for bit in encoded_bits:
        if(bit==''):
            break
        else:
            decoded_character_code = decoded_character_code + str(bit)
            if decoded_character_code in bit_codes_dict.values():
                #get key of the decoded code
                character = list(bit_codes_dict.keys())[list(bit_codes_dict.values()).index(decoded_character_code)]
                decompressed_file.write(character)
                decoded_character_code = ''
    
    decompressed_file.close()    

In [33]:
comp_f = open("compressed_file.bin",'rb')
decompress(comp_f,huf.file.tree.new_codes)

b'0001110001001110010011101001100001110101100101110111110110111110'


In [24]:
def print_compression_table():

SyntaxError: unexpected EOF while parsing (<ipython-input-24-c21ebc0bc847>, line 1)